# API Project

https://www.opendata.nhs.scot/dataset/weekly-accident-and-emergency-activity-and-waiting-times/resource/a5f7ca94-c810-41b5-a7c9-25c18d43e5a4

### Import Libraries

In [19]:
import requests
import json
from sqlalchemy import create_engine  # For connecting to a PostgreSQL database (or other databases)
import os
import openpyxl  # For reading and writing Excel (.xlsx) files

### Make the API request

In [11]:
# API URL
url = "https://www.opendata.nhs.scot/api/3/action/datastore_search"

params = {
    "resource_id": "a5f7ca94-c810-41b5-a7c9-25c18d43e5a4",
    "limit": 1300,
    "filters": json.dumps({"TreatmentLocation": "S308H"})
}

# Send the request
response = requests.get(url, params=params)

### Interpret the result

In [12]:
if response.status_code == 200:
    print(f"Request was a success! Status code: {response.status_code}")
else:
    print(f"Error: {response.status_code}")

Request was a success! Status code: 200


### Print the records

In [13]:
if response.status_code == 200:
    data = response.json()
    records = data.get("result", {}).get("records", [])  # Extract the records

### Add records to a data frame

In [14]:
import pandas as pd

# Convert records to a DataFrame
df = pd.DataFrame(records)

# Show the first few rows
df.tail()

,_id,WeekEndingDate,Country,HBT,TreatmentLocation,DepartmentType,AttendanceCategory,NumberOfAttendancesEpisode,NumberWithin4HoursEpisode,NumberOver4HoursEpisode,PercentageWithin4HoursEpisode,NumberOver8HoursEpisode,PercentageOver8HoursEpisode,NumberOver12HoursEpisode,PercentageOver12HoursEpisode
1087,35682,20250223,S92000003,S08000024,S308H,Type 1,Unplanned,968,595,373,61.5,116,12.0,69,7.1
1088,35683,20250223,S92000003,S08000024,S308H,Type 1,All,1146,773,373,67.5,116,10.1,69,6.0
1089,35768,20250302,S92000003,S08000024,S308H,Type 1,New planned,199,199,0,100.0,0,0.0,0,0.0
1090,35769,20250302,S92000003,S08000024,S308H,Type 1,Unplanned,1078,674,404,62.5,114,10.6,50,4.6
1091,35770,20250302,S92000003,S08000024,S308H,Type 1,All,1277,873,404,68.4,114,8.9,50,3.9


# Transformation

In [15]:
df.columns = df.columns.str.lower()
df = df.drop(columns = ["country", "hbt"])
df.head()

,_id,weekendingdate,treatmentlocation,departmenttype,attendancecategory,numberofattendancesepisode,numberwithin4hoursepisode,numberover4hoursepisode,percentagewithin4hoursepisode,numberover8hoursepisode,percentageover8hoursepisode,numberover12hoursepisode,percentageover12hoursepisode
0,53,20150222,S308H,Type 1,Unplanned,972,903,69,92.9,2,0.2,0,0.0
1,54,20150222,S308H,Type 1,All,972,903,69,92.9,2,0.2,0,0.0
2,117,20150301,S308H,Type 1,Unplanned,1011,917,94,90.7,4,0.4,0,0.0
3,118,20150301,S308H,Type 1,All,1011,917,94,90.7,4,0.4,0,0.0
4,181,20150308,S308H,Type 1,Unplanned,1040,981,59,94.3,5,0.5,0,0.0


# Add data to the database

### Create connection to the database

In [16]:
# Create a connection to the database
db_user = "postgres"
db_password = "Chilli55"
db_host = "localhost"
db_port = "5432"
db_name = "A&E Attendances"

engine = create_engine(f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")

with engine.connect() as conn:
    print("Connected to PostgreSQL successfully!")

Connected to PostgreSQL successfully!


### Load data into the database

In [18]:
existing_ids = pd.read_sql("SELECT _id FROM a_and_e_attendances", con=engine)['_id']
new_data = df[~df['_id'].isin(existing_ids)]

if not new_data.empty:
    new_data.to_sql("a_and_e_attendances", con=engine, if_exists="append", index=False)
    print(f"{len(new_data)} row(s) have been added to the database.")
else:
    print("Data already present within table.")

print("All data processed successfully!")

Data already present within table.
All data processed successfully!


### Load the data into Excel

In [24]:
df.to_excel("a_and_e_attendances.xlsx", index = False)
print(f"{len(df)} row (s) added to Excel sheet.")

PermissionError: [Errno 13] Permission denied: 'a_and_e_attendances.xlsx'

Next Steps
  
- Also save the data in Excel.
- Automate API calls (e.g., run it on a schedule and store data in a database).
- Create a dashboard - potentially pull all information in and add lookup tables to SQL for health board and hospital or add in data on demographics and or waits.